In [80]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime
from time import *


df = pd.read_csv('/home/apprenant/Desktop/FARIZD/taxi_nyc/taxi_nyc/Data/row_data/train.csv')
df = df.sample(n=100000)

df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1265168,id0579794,2,2016-03-29 10:19:59,2016-03-29 10:48:25,1,-73.863800,40.769650,-73.970596,40.755638,N,1706
975664,id0254421,1,2016-02-22 07:54:00,2016-02-22 07:59:56,1,-73.960602,40.761341,-73.963280,40.770988,N,356
546126,id3441339,2,2016-06-18 00:58:40,2016-06-18 01:53:07,1,-73.989891,40.734489,-73.944206,40.779499,N,3267
44127,id2316427,2,2016-03-16 11:26:23,2016-03-16 11:28:33,1,-73.987068,40.771255,-73.982033,40.777679,N,130
94164,id3210030,1,2016-04-01 14:22:57,2016-04-01 14:31:09,1,-73.939476,40.805206,-73.943741,40.793118,N,492


In [81]:
df.shape

(100000, 11)

In [82]:
df.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

In [83]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format="%Y-%m-%d %H:%M:%S")
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'], format="%Y-%m-%d %H:%M:%S")


In [84]:
df.dtypes


id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                  int64
dtype: object

In [85]:
#df['trip_duration']= df['trip_duration'].apply(lambda x :strftime('%H:%M:%S', gmtime(x)))

In [86]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1265168,id0579794,2,2016-03-29 10:19:59,2016-03-29 10:48:25,1,-73.863800,40.769650,-73.970596,40.755638,N,1706
975664,id0254421,1,2016-02-22 07:54:00,2016-02-22 07:59:56,1,-73.960602,40.761341,-73.963280,40.770988,N,356
546126,id3441339,2,2016-06-18 00:58:40,2016-06-18 01:53:07,1,-73.989891,40.734489,-73.944206,40.779499,N,3267
44127,id2316427,2,2016-03-16 11:26:23,2016-03-16 11:28:33,1,-73.987068,40.771255,-73.982033,40.777679,N,130
94164,id3210030,1,2016-04-01 14:22:57,2016-04-01 14:31:09,1,-73.939476,40.805206,-73.943741,40.793118,N,492


In [87]:
df.isnull().sum()


id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [88]:
df.duplicated().sum()

0

In [89]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [90]:
df['distance'] = haversine(df.pickup_latitude, df.pickup_longitude, df.dropoff_latitude, df.dropoff_longitude)

In [91]:
df["distance"] = df["distance"].round(3)

In [92]:
def vitesse_trajet(distance, temps ):
    """Retourne la vitesse entre les le début et la fin du trajet."""
    res = (distance * 1000)/ temps
    kmh = res * 3.6 
    return kmh

In [93]:
df['vitesse_moyenne'] = vitesse_trajet(df.distance, df.trip_duration)

In [94]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,vitesse_moyenne
1265168,id0579794,2,2016-03-29 10:19:59,2016-03-29 10:48:25,1,-73.863800,40.769650,-73.970596,40.755638,N,1706,9.128,19.261899
975664,id0254421,1,2016-02-22 07:54:00,2016-02-22 07:59:56,1,-73.960602,40.761341,-73.963280,40.770988,N,356,1.096,11.083146
546126,id3441339,2,2016-06-18 00:58:40,2016-06-18 01:53:07,1,-73.989891,40.734489,-73.944206,40.779499,N,3267,6.313,6.956474
44127,id2316427,2,2016-03-16 11:26:23,2016-03-16 11:28:33,1,-73.987068,40.771255,-73.982033,40.777679,N,130,0.831,23.012308
94164,id3210030,1,2016-04-01 14:22:57,2016-04-01 14:31:09,1,-73.939476,40.805206,-73.943741,40.793118,N,492,1.391,10.178049


In [95]:
df['weekday']=df.pickup_datetime.dt.weekday
df['weekday'].value_counts()

4    15425
5    15117
3    14936
2    14390
1    14025
6    13304
0    12803
Name: weekday, dtype: int64

In [96]:
df['weekday']=df.pickup_datetime.dt.day_name()

In [97]:
df.groupby('weekday')['distance'].sum()

weekday
Friday       52357.862
Monday       46225.955
Saturday     50298.977
Sunday       48498.464
Thursday     51713.258
Tuesday      46717.501
Wednesday    47820.020
Name: distance, dtype: float64

In [98]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,vitesse_moyenne,weekday
1265168,id0579794,2,2016-03-29 10:19:59,2016-03-29 10:48:25,1,-73.863800,40.769650,-73.970596,40.755638,N,1706,9.128,19.261899,Tuesday
975664,id0254421,1,2016-02-22 07:54:00,2016-02-22 07:59:56,1,-73.960602,40.761341,-73.963280,40.770988,N,356,1.096,11.083146,Monday
546126,id3441339,2,2016-06-18 00:58:40,2016-06-18 01:53:07,1,-73.989891,40.734489,-73.944206,40.779499,N,3267,6.313,6.956474,Saturday
44127,id2316427,2,2016-03-16 11:26:23,2016-03-16 11:28:33,1,-73.987068,40.771255,-73.982033,40.777679,N,130,0.831,23.012308,Wednesday
94164,id3210030,1,2016-04-01 14:22:57,2016-04-01 14:31:09,1,-73.939476,40.805206,-73.943741,40.793118,N,492,1.391,10.178049,Friday
